Creamos los modelos por tipo de cliente

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor 

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once') 

In [5]:
#from google.colab import files
#uploaded = files.upload() 

Saving casual_encoded.csv to casual_encoded.csv


In [3]:
df_casual = pd.read_csv("Datos/casual_encoded.csv", index_col = 0)
df_casual.head()

,dteday,temp_es,windspeed_es,casual_es,dia_sem_encoded,laborables_0,laborables_1,weathersit_encoded,month_encoded,year_0,year_1
0,01-01-2018,-0.827613,-0.387833,-0.755455,4,1,0,2,1,1,0
1,02-01-2018,-0.722069,0.748899,-1.046996,3,0,1,2,1,1,0
2,03-01-2018,-1.635432,0.745931,-1.063031,1,0,1,3,1,1,0
3,04-01-2018,-1.615560,-0.389769,-1.080523,1,0,1,3,1,1,0
4,05-01-2018,-1.468226,-0.046477,-1.118424,0,0,1,3,1,1,0


In [8]:
#from google.colab import files
#uploaded = files.upload() 

Saving registered_encoded.csv to registered_encoded.csv


In [4]:
df_registered = pd.read_csv("Datos/registered_encoded.csv", index_col = 0)
df_registered.head()

,dteday,temp_es,windspeed_es,registered_es,dia_sem_encoded,laborables_0,laborables_1,weathersit_encoded,month_encoded,year_0,year_1
0,01-01-2018,-0.827613,-0.387833,-1.927745,1,1,0,2,1,1,0
1,02-01-2018,-0.722069,0.748899,-1.917480,1,0,1,2,1,1,0
2,03-01-2018,-1.635432,0.745931,-1.558846,2,0,1,3,1,1,0
3,04-01-2018,-1.615560,-0.389769,-1.414494,3,0,1,3,1,1,0
4,05-01-2018,-1.468226,-0.046477,-1.373434,3,0,1,3,1,1,0


In [5]:
X = df_casual.drop(["casual_es", "dteday"], axis = 1)
y = df_casual["casual_es"]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
arbol = DecisionTreeRegressor(random_state =0)

arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [8]:
#fig = plt.figure(figsize = (10,6))
#tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
#plt.show()

In [9]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.0

In [10]:
print(arbol.tree_.max_depth)

19


In [13]:
param = {"max_depth": [2,4,6], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100],
        "random_state":[0]} 

In [14]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [16]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100],
                         'random_state': [0]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [18]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10, random_state=0)

In [19]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [20]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [21]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,0.434915,0.385741,0.621080,0.622140,test,Decision tree II
1,0.442333,0.395004,0.628493,0.602544,train,Decision tree II
